In [5]:
import cfp
import scanpy as sc
import anndata as ad
from cfp.external import CFJaxSCVI
import numpy as np

In [2]:
split = 5
model_name = "vital-dragon-422"

In [12]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_{split}.h5ad"
adata_train = sc.read_h5ad(adata_train_path)
adata_test = sc.read_h5ad(adata_test_path)
adata_ood = sc.read_h5ad(adata_ood_path)
    

In [13]:
vae = CFJaxSCVI.load("/lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/32_1024", adata=adata_train)
adata_train.obsm["X_scVI"] = vae.get_latent_representation(adata_train)
adata_test.obsm["X_scVI"] = vae.get_latent_representation(adata_test)
adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood)


INFO     File /lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/32_1024/model.pt already downloaded


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/model/base/_utils.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be a

INFO     Jax module moved to cuda:0.Note: Pytorch lightning will show GPU is not being used for the Trainer.       


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
GPU available: True (cuda), used: False
T

Epoch 1/18:   6%|▌         | 1/18 [00:01<00:30,  1.77s/it, v_num=1, train_loss_step=402, train_loss_epoch=402]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 1/18:   6%|▌         | 1/18 [00:01<00:30,  1.77s/it, v_num=1, train_loss_step=402, train_loss_epoch=402]
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [21]:
adata_ood_ctrl = adata_ood[adata_ood.obs["condition"].str.contains("Vehicle")]
adata_test_ctrl = adata_test[adata_test.obs["condition"].str.contains("Vehicle")]
adata_ood_ctrl.obs["control"] = True
adata_test_ctrl.obs["control"] = True
covariate_data_ood = adata_ood[~adata_ood.obs["condition"].str.contains("Vehicle")].obs.drop_duplicates(subset=["condition"])
covariate_data_test = adata_test[~adata_test.obs["condition"].str.contains("Vehicle")].obs.drop_duplicates(subset=["condition"])

/tmp/ipykernel_651063/2979151925.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_ood_ctrl.obs["control"] = True
/tmp/ipykernel_651063/2979151925.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_test_ctrl.obs["control"] = True


In [16]:
cf = cfp.model.CellFlow.load(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/out/{model_name}_CellFlow.pkl")

In [17]:
adata_ood_ctrl

AnnData object with n_obs × n_vars = 0 × 2001
    obs: 'cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score', 'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2', 'product_dose', 'product_name', 'proliferation_index', 'replicate', 'size_factor', 'target', 'vehicle', 'perturbation', 'drug', 'cell_line', 'logdose', 'condition', 'n_genes', 'pubchem_name', 'pubchem_ID', 'smiles', 'control', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'ood_5', 'split', '_scvi_batch'
    uns: 'cell_line_dict', 'ecfp_dict', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'ecfp', 'X_scVI'

In [20]:
adata_ood_ctrl

AnnData object with n_obs × n_vars = 0 × 2001
    obs: 'cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score', 'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2', 'product_dose', 'product_name', 'proliferation_index', 'replicate', 'size_factor', 'target', 'vehicle', 'perturbation', 'drug', 'cell_line', 'logdose', 'condition', 'n_genes', 'pubchem_name', 'pubchem_ID', 'smiles', 'control', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'ood_5', 'split', '_scvi_batch'
    uns: 'cell_line_dict', 'ecfp_dict', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'ecfp', 'X_scVI'

In [22]:
preds_ood = cf.predict(adata=adata_ood_ctrl, sample_rep="X_scVI", condition_id_key="condition", covariate_data=covariate_data_ood)

/ictstr01/home/icb/dominik.klein/git_repos/cell_flow_perturbation/src/cfp/data/_datamanager.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _covariate_data["cell_index"] = _covariate_data.index
100%|██████████| 24/24 [00:00<00:00, 660.69it/s]


In [23]:
preds_test = cf.predict(adata=adata_test_ctrl, sample_rep="X_scVI", condition_id_key="condition", covariate_data=covariate_data_test)

/ictstr01/home/icb/dominik.klein/git_repos/cell_flow_perturbation/src/cfp/data/_datamanager.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _covariate_data["cell_index"] = _covariate_data.index
100%|██████████| 659/659 [00:00<00:00, 666.36it/s]


KeyboardInterrupt: 

In [29]:
import pandas as pd
all_data = []
conditions = []

for condition, array in preds_ood.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_ood_result = ad.AnnData(X=np.empty((len(all_data_array),2001)), obs=obs_data)
adata_ood_result.obsm["X_scVI_pred"] = all_data_array

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [30]:
adata_ood_result.layers["pred_reconstruction"] = vae.get_reconstructed_expression(adata_ood_result, use_rep="X_scVI_pred")

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [31]:
adata_ood_result.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/sciplex/adata_ood_with_predictions_{split}.h5ad")


In [25]:
preds_ood.keys()

dict_keys(['A549_Alvespimycin_(17-DMAG)_HCl_10.0', 'A549_Alvespimycin_(17-DMAG)_HCl_100.0', 'A549_Belinostat_(PXD101)_10.0', 'A549_Belinostat_(PXD101)_100.0', 'A549_Belinostat_(PXD101)_1000.0', 'A549_Dacinostat_(LAQ824)_10.0', 'A549_Dacinostat_(LAQ824)_100.0', 'A549_Dacinostat_(LAQ824)_1000.0', 'A549_Flavopiridol_HCl_10.0', 'A549_Flavopiridol_HCl_100.0', 'A549_Flavopiridol_HCl_1000.0', 'A549_Flavopiridol_HCl_10000.0', 'A549_Givinostat_(ITF2357)_10.0', 'A549_Givinostat_(ITF2357)_100.0', 'A549_Givinostat_(ITF2357)_1000.0', 'A549_Givinostat_(ITF2357)_10000.0', 'A549_Hesperadin_10.0', 'A549_Hesperadin_100.0', 'A549_Hesperadin_1000.0', 'A549_Quisinostat_(JNJ-26481585)_2HCl_10.0', 'A549_Quisinostat_(JNJ-26481585)_2HCl_100.0', 'A549_TAK-901_10.0', 'A549_TAK-901_100.0', 'A549_TAK-901_1000.0', 'A549_Tanespimycin_(17-AAG)_10.0', 'A549_Tanespimycin_(17-AAG)_100.0', 'A549_Tanespimycin_(17-AAG)_1000.0', 'K562_Alvespimycin_(17-DMAG)_HCl_10.0', 'K562_Alvespimycin_(17-DMAG)_HCl_100.0', 'K562_Alvespimy

In [33]:
adata_ood.obsm

AxisArrays with keys: X_pca, ecfp, X_scVI

In [36]:
adata_ood.layers["X_scvi_recon"] = vae.get_reconstructed_expression(adata_ood, use_rep="X_scVI")

In [38]:
from sklearn.metrics import r2_score
r2_score(np.array(np.mean(adata_ood.X, axis=0))[0,:], np.mean(adata_ood.layers["X_scvi_recon"], axis=0))

0.9942914843559265

In [39]:
adata_ood

AnnData object with n_obs × n_vars = 20173 × 2001
    obs: 'cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score', 'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2', 'product_dose', 'product_name', 'proliferation_index', 'replicate', 'size_factor', 'target', 'vehicle', 'perturbation', 'drug', 'cell_line', 'logdose', 'condition', 'n_genes', 'pubchem_name', 'pubchem_ID', 'smiles', 'control', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'ood_5', 'split', '_scvi_batch'
    uns: 'cell_line_dict', 'ecfp_dict', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'ecfp', 'X_scVI'
    layers: 'X_scvi_recon'

In [53]:
for cond in adata_ood.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    r2_true_recon = r2_score(np.array(np.mean(adata_ood[adata_ood.obs["condition"]==cond].X, axis=0))[0,:], np.mean(adata_ood[adata_ood.obs["condition"]==cond].layers["X_scvi_recon"], axis=0))
    r2_true_pred = r2_score(np.array(np.mean(adata_ood[adata_ood.obs["condition"]==cond].X, axis=0))[0,:], np.mean(adata_ood_result[adata_ood_result.obs["condition"]==cond].layers["pred_reconstruction"], axis=0))
    r2_recon_pred = r2_score(np.array(np.mean(adata_ood[adata_ood.obs["condition"]==cond].layers["X_scvi_recon"], axis=0)), np.mean(adata_ood_result[adata_ood_result.obs["condition"]==cond].layers["pred_reconstruction"], axis=0))
    
    print(r2_true_recon, r2_true_pred, r2_recon_pred)
    

0.984657883644104 0.981431245803833 0.9981325268745422
0.9278574585914612 0.9028077125549316 0.9877969026565552
0.9846107363700867 0.9814658761024475 0.9989372491836548
0.9712024331092834 0.9645401835441589 0.9977645874023438
0.9044577479362488 0.858910858631134 0.9798272252082825
0.919241189956665 0.8727970123291016 0.9793018102645874
0.9285982847213745 0.676747739315033 0.80793297290802
0.9767060875892639 0.2097795605659485 0.24115145206451416
0.9784423112869263 0.9707511067390442 0.9954734444618225
0.8981156945228577 0.7757612466812134 0.9293370842933655
0.9841639399528503 0.9660423994064331 0.9924464225769043
0.9862672090530396 0.9713453054428101 0.9919509887695312
0.9786919951438904 0.9772029519081116 0.9996663928031921
0.9683473706245422 0.9675544500350952 0.9995837807655334
0.9241098761558533 0.8748791217803955 0.9806903004646301
0.9635095596313477 0.22204184532165527 0.26591211557388306
0.9832130074501038 0.9614953994750977 0.9860519170761108
0.9818530678749084 0.96176511049270

In [45]:
# true, reconc
np.mean([r2_score(np.array(np.mean(adata_ood[adata_ood.obs["condition"]==cond].X, axis=0))[0,:], np.mean(adata_ood[adata_ood.obs["condition"]==cond].layers["X_scvi_recon"], axis=0)) for cond in adata_ood.obs["condition"].cat.categories])
    

0.9554968774318695

In [50]:
# true, pred
np.mean([r2_score(np.array(np.mean(adata_ood[adata_ood.obs["condition"]==cond].X, axis=0))[0,:], np.mean(adata_ood_result[adata_ood_result.obs["condition"]==cond].layers["pred_reconstruction"], axis=0)) for cond in adata_ood_result.obs["condition"].cat.categories])

0.8361269637762782

In [55]:
# recon, pred
np.mean([r2_score(np.array(np.mean(adata_ood[adata_ood.obs["condition"]==cond].layers["X_scvi_recon"], axis=0)), np.mean(adata_ood_result[adata_ood_result.obs["condition"]==cond].layers["pred_reconstruction"], axis=0)) for cond in adata_ood_result.obs["condition"].cat.categories])

0.8972270251756691

In [40]:
adata_ood.obs

,cell_type,dose,dose_character,dose_pattern,g1s_score,g2m_score,pathway,pathway_level_1,pathway_level_2,product_dose,...,pubchem_ID,smiles,control,ood_1,ood_2,ood_3,ood_4,ood_5,split,_scvi_batch
index,,,,,,,,,,,,,,,,,,,,,
A01_E09_RT_BC_108_Lig_BC_256-0-0,A549,10.0,10,4,0.730612,1.148259,Epigenetics,Epigenetic regulation,Histone deacetylation,Quisinostat (JNJ-26481585) 2HCl_10,...,122129987,CN1C=C(C2=CC=CC=C21)CNCC3CCN(CC3)C4=NC=C(C=N4)...,False,not ood,not ood,not ood,A549_Quisinostat_(JNJ-26481585)_2HCl_10.0,A549_Quisinostat_(JNJ-26481585)_2HCl_10.0,ood,0
A01_E09_RT_BC_116_Lig_BC_193-0-0,A549,10.0,10,4,1.494859,0.000000,Cytoskeletal Signaling,Protein folding & Protein degradation,HSP90 activity,Alvespimycin (17-DMAG) HCl_10,...,9852573,CC1CC(C(C(C=C(C(C(C=CC=C(C(=O)NC2=CC(=O)C(=C(C...,False,not ood,not ood,not ood,A549_Alvespimycin_(17-DMAG)_HCl_10.0,A549_Alvespimycin_(17-DMAG)_HCl_10.0,ood,0
A01_E09_RT_BC_129_Lig_BC_15-0-0,A549,10000.0,10000,1,1.767667,1.535335,Cell Cycle,Cell cycle regulation,Cell cycle regulation,Flavopiridol HCl_10000,...,9910986,CN1CCC(C(C1)O)C2=C(C=C(C3=C2OC(=CC3=O)C4=CC=CC...,False,A549_Flavopiridol_HCl_10000.0,not ood,not ood,not ood,A549_Flavopiridol_HCl_10000.0,ood,0
A01_E09_RT_BC_136_Lig_BC_372-0-0,A549,1000.0,1000,2,1.063902,1.063902,Epigenetics,Epigenetic regulation,Histone deacetylation,Belinostat (PXD101)_1000,...,6918638,C1=CC=C(C=C1)NS(=O)(=O)C2=CC=CC(=C2)C=CC(=O)NO,False,not ood,A549_Belinostat_(PXD101)_1000.0,not ood,not ood,A549_Belinostat_(PXD101)_1000.0,ood,0
A01_E09_RT_BC_136_Lig_BC_6-0-0,A549,1000.0,1000,2,0.402133,0.688145,Cell Cycle,Cell cycle regulation,Aurora kinase activity,Hesperadin_1000,...,135421442,CCS(=O)(=O)NC1=CC2=C(C=C1)NC(=C2C(=NC3=CC=C(C=...,False,not ood,A549_Hesperadin_1000.0,not ood,not ood,A549_Hesperadin_1000.0,ood,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H12_E09_RT_BC_353_Lig_BC_331-1,K562,0.0,0,1,1.842164,1.296121,NA,Vehicle,Vehicle,Vehicle_0,...,NaN,NaN,True,not ood,not ood,not ood,not ood,not ood,test,0
H12_E09_RT_BC_85_Lig_BC_284-1,K562,0.0,0,3,1.677900,1.478572,NA,Vehicle,Vehicle,Vehicle_0,...,NaN,NaN,True,not ood,not ood,not ood,not ood,not ood,test,0
H12_F10_RT_BC_151_Lig_BC_60-1,K562,0.0,0,1,1.440329,0.959749,NA,Vehicle,Vehicle,Vehicle_0,...,NaN,NaN,True,not ood,not ood,not ood,not ood,not ood,test,0


In [13]:
all_data = []
conditions = []

for condition, array in preds_test.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_test_result = ad.AnnData(X=np.empty((len(all_data_array),2000)), obs=obs_data)
adata_test_result.obsm["X_pca_pred"] = all_data_array                       

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [14]:
cfpp.reconstruct_pca(query_adata=adata_test_result, use_rep="X_pca_pred", ref_adata=adata_train, layers_key_added="X_recon_pred")

In [15]:
adata_ood_result.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/combosciplex/adata_ood_with_predictions_{split}.h5ad")
adata_test_result.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/combosciplex/adata_test_with_predictions_{split}.h5ad")